In [141]:
####토큰 수 통계
print(df["token_count"].describe())

count      450.000000
mean      1782.420000
std       1554.217313
min         75.000000
25%        800.000000
50%       1601.000000
75%       2222.000000
max      21531.000000
Name: token_count, dtype: float64


#### Dataload &  df 준비

In [138]:
import os
from dotenv import load_dotenv
import json
import pandas as pd

# .env 파일 로드
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

file_path = "../final.json"

# JSON 파일 로드
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.DataFrame(data)
df =df[['title','link','date','content','source']]
df["date"] = pd.to_datetime(df["date"])
df['quarter'] = df["date"].dt.to_period("Q")

#### 토큰수 구하기

In [140]:
import tiktoken

# OpenAI GPT-3.5 토크나이저
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
# 각 기사의 토큰 개수 계산
df["token_count"] = df["content"].apply(lambda x: len(enc.encode(x)))

In [103]:
import re
from tqdm import tqdm

tqdm.pandas()

# 광고 및 출처 정보 제거 정규식 패턴
AD_PATTERNS = [
    r"责任编辑：.*?",
    r"（本文来源.*?）",
    r"点击阅读全文.*?",
    r"来源：.*?",
    r"更多精彩内容.*?",
    r"本文转载自.*?",
    r"欢迎关注我们的.*?",
    r"广告.*?",
    r"（.*?记者.*?报道）",
    r"如需转载请注明.*?",
    r"查看更多相关信息.*?"
]

def clean_text(text):
    """불필요한 광고 및 출처 제거하고, 구두점 유지"""
    if not isinstance(text, str):
        return ""

    # 광고 및 출처 제거
    for pattern in AD_PATTERNS:
        text = re.sub(pattern, "", text)

    # 특수 문자 및 공백 정리 (단, 구두점 `。！？`는 유지)
    text = re.sub(r'[^\w\s。！？]', '', text)  # 한자, 숫자, 구두점 외 제거
    text = re.sub(r'\s+', ' ', text).strip()  # 공백 정리

    return text

# def sliding_window(text: str, window_size: int, stride: int):
#     windows = []
#     start = 0
#     while start < len(text):
#         windows.append(text[start:start + window_size])
#         start += stride
#     return windows
#
# def get_window_size_and_stride(text_length):
#     """텍스트 길이에 따라 윈도우 크기와 슬라이딩 간격을 동적으로 조절"""
#     if text_length > 10000:
#         return 3000, 1500  # 긴 문서 → 큰 윈도우, 50% 겹침
#     elif text_length > 5000:
#         return 2000, 1000  # 중간 문서 → 중간 윈도우, 50% 겹침
#     else:
#         return 1000, 500  # 짧은 문서 → 작은 윈도우, 50% 겹침
#
# df['input_text'] = df.progress_apply(
#     lambda row: sliding_window(clean_text(row['content']), *get_window_size_and_stride(len(clean_text(row['content']))))
#     if row['token_count'] > 3500 else clean_text(row['content']),
#     axis=1
# )

df['input_text'] = df['content'].apply(clean_text)

In [142]:
df

,title,link,date,content,source,quarter,token_count
0,朝鲜要用150天救经济摆脱美日韩经济封锁,https://world.huanqiu.com/article/9CaKrnJlXnI,1970-01-01 00:20:42.777600,自上个月朝鲜宣布发射人造卫星以来，美国、日本和韩国高举制裁大棒，对朝鲜的贸易活动进行了进一步...,环球网,1970Q1,1413
1,抢抓新一轮西部大开发战略机遇 全力推进宁夏招商引资新跨越_光明网,https://topics.gmw.cn/2010-10/28/content_13423...,1970-01-01 00:21:28.224000,三、深入实施西部大开发战略，为宁夏招商引资明确了新任务\n今后10年是深入实施西部大开发战略...,光明网,1970Q1,3577
2,日本外相担忧朝铀浓缩称美日韩应加强情报合作 - 国际新闻,https://world.huanqiu.com/article/9CaKrnJpmLX,1970-01-01 00:21:30.384000,环球网记者李宗泽报道，据韩联社11月22日消息，日本外相前原诚司22日在外务省会见美国国务院...,环球网,1970Q1,313
3,日媒称日美韩外长会谈将发表联合声明对朝施压 - 国际新闻,https://world.huanqiu.com/article/9CaKrnJpxsJ,1970-01-01 00:21:31.593600,环球网记者仲伟东12月6日报道，日本共同社6日报道称，美日韩三国外长将于美国当地时间6日下午...,环球网,1970Q1,464
4,美国吁美日韩三国军演处理半岛危机欲把责任推给中国,http://world.huanqiu.com/article/9CaKrnJpAeY,1970-01-01 00:21:31.852800,美国处理目前半岛危机的战术意图越来越清晰：把责任推给中国、把军演进行到底。\n已是完成时的被...,环球网,1970Q1,2219
...,...,...,...,...,...,...,...
445,安理会召开紧急会议日媒：美日韩力争构筑对朝包围网,https://mil.huanqiu.com/article/9CaKrnK18sZ,NaT,【环球网报道 记者 王欢】联合国安理会当地时间3月8日上午就朝鲜发射弹道导弹召开紧急闭门会议...,环球网,NaT,699
446,和评理｜美日韩沆瀣一气，戴维营峰会危及地区稳定,https://3w.huanqiu.com/a/72d6dc/4E8d7GVNmpE?ag...,NaT,美日韩三国领导人将于8月18日在华盛顿近郊的戴维营召开峰会。有报道称，美国总统拜登预计将就延...,环球网,NaT,736
447,美日韩“总长”将会面对朝情报共享是重点 - 环球军事 - 环球网,https://mil.huanqiu.com/article/9CaKrnJFa1N,NaT,【环球军事报道】据韩国YTN电视台1日报道，韩国联合参谋本部议长崔润喜、美军参谋长联席会议主...,环球网,NaT,1086
448,日韩关系遇冷日媒敦促安倍转变历史问题态度,https://m.huanqiu.com/article/9CaKrnJDOmI,NaT,日本首相安倍晋三\n【环球网综合报道】日本电视台1月3日指出，日本首相安倍晋三悍然参拜靖国神...,环球网,NaT,989


#### LangChain을 이용한 감성 분석 프롬프트

In [123]:
import os
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

# 환경 변수 로드 (API KEY 설정)
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI LLM 모델 설정
llm = ChatOpenAI(openai_api_key=api_key, model_name="gpt-4-turbo-preview", temperature=0)

# 감성 분석 기본 프롬프트
prompt_template = PromptTemplate(
    input_variables=["input_text"],
    template=(
        "다음 중국어 기사의 감성을 -5에서 +5 사이의 정수로 평가하세요. "
        "설명 없이 숫자만 출력하세요.\n\n"
        "문장: {input_text}\n"
        "답변:"
    )
)

#### Few-Shot Prompt 및 템플릿 설정

In [115]:
# few_shot_prompt = FewShotPromptTemplate(
#     examples=[
#         {"input_text": "美国总统国家安全事务助理沙利文2日与到访的日本国家安全保障局局长北村滋和韩国国家安保室长徐薰举行会晤，讨论美国对朝鲜政策评估及其他问题。", "score": "0"},
#         {"input_text": "外交部发言人刘为民在新闻发布会上说，中国认为国际社会、尤其是亚太国家应当作出建设性的努力，维护朝鲜半岛和东北亚的和平与稳定，而不是相反。", "score": "-3"},
#         {"input_text": "中方坚决反对操弄集团政治，坚决反对任何挑起和激化矛盾、损害他国战略安全和利益的言行，坚决反对在亚太地区搞封闭排他的“小圈子”。", "score": "-5"},
#         {"input_text": "全国道路交通安全形势总体平稳，未接报重特大道路交通事故，全国主干公路通行有序。", "score": "1"},
#         {"input_text": "中国人民银行表示，双方再次续签双边本币互换协议，有助于进一步深化两国货币金融合作，促进双边贸易和投资便利化，维护金融市场稳定。", "score": "3"},
#         {"input_text": "展望未来，国际社会共同推动构建人类命运共同体，守望相助、互利共赢，必将创造一个共同繁荣的美好未来，为各国人民带来更大福祉。", "score": "5"}
#     ],
#     example_prompt=PromptTemplate(
#         input_variables=["input_text", "score"],
#         template="{score}"
#     ),
#     prefix=(
#         "다음 중국어 기사의 감성을 -5에서 +5 사이 정수로 평가하세요. "
#         "숫자 외에는 아무것도 출력하지 마세요. "
#         "설명, 기호, 문장 부호 없이 숫자 하나만 출력하세요. "
#     ),
#     suffix="답변:",
#     input_variables=["input_text"]
# )
# # 감성 분석 체인 구성
# sentiment_chain = few_shot_prompt | llm

In [124]:
# 감성 분석 실행 테스트
input_text = "整个国家陷入绝望，经济完全崩溃，人民生活在痛苦和恐惧之中。"
formatted_prompt = prompt_template.format(input_text=input_text)
response = llm.invoke(formatted_prompt)

In [131]:
def analyze_sentiment(text):
    try:
        formatted_prompt = prompt_template.format(input_text=text)
        response = llm.invoke(formatted_prompt)
        return int(response.content.strip())  # 숫자로 변환
    except Exception as e:
        print(f"에러 : {e}")
        return None


In [137]:
from concurrent.futures import ThreadPoolExecutor

backup_file = "sentiment_analysis_backup.csv"

# 기존 df가 이미 존재하는 경우, 백업 파일이 없으면 저장
if not os.path.exists(backup_file):
    df.to_csv(backup_file, index=False)
    print(f"✅ 백업 파일이 없어서 새로 저장함: {backup_file}")
else:
    print("✅ 기존 백업 파일이 존재함.")

if "sentiment_score" not in df.columns:
    df["sentiment_score"] = None  # 컬럼 추가
    print("✅ 'sentiment_score' 컬럼이 없어서 추가함.")

incomplete_rows = df[df["sentiment_score"].isna()]
# 병렬 처리 실행
def process_row(index, text):
    score = analyze_sentiment(text)
    df.at[index, "sentiment_score"] = score  # 결과 업데이트
    return index

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = {executor.submit(process_row, idx, row["input_text"]): idx for idx, row in incomplete_rows.iterrows()}

    for i, future in enumerate(futures):
        future.result()  # 실행 완료 대기
        if (i + 1) % 5 == 0:  # 5개 완료될 때마다 저장
            df.to_csv(backup_file, index=False)
            print(f"🔄 {i + 1}개 처리 완료, 데이터 저장!")

# 마지막 데이터 저장
df.to_csv(backup_file, index=False)
print("✅ 모든 데이터 처리 완료 및 저장됨!")

✅ 백업 파일이 없어서 새로 저장함: sentiment_analysis_backup.csv
✅ 'sentiment_score' 컬럼이 없어서 추가함.
🔄 5개 처리 완료, 데이터 저장!
🔄 10개 처리 완료, 데이터 저장!
🔄 15개 처리 완료, 데이터 저장!
에러 : Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4-turbo-preview in organization org-A1JNNlkMXNItEVSFqRa3W3Ch on tokens per min (TPM): Limit 30000, Used 29028, Requested 1186. Please try again in 428ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
에러 : Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4-turbo-preview in organization org-A1JNNlkMXNItEVSFqRa3W3Ch on tokens per min (TPM): Limit 30000, Used 29647, Requested 634. Please try again in 562ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
에러 : Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4-turbo-preview in organization org-A1JNNlkMXNItEVSFqRa3

In [127]:
df_test = df.head(3).copy()
df_test["sentiment_score"] = df_test["input_text"].apply(analyze_sentiment)

In [128]:
df_test

,title,link,date,content,source,quarter,token_count,input_text,sentiment_scores,sentiment_score
0,抢抓新一轮西部大开发战略机遇 全力推进宁夏招商引资新跨越_光明网,https://topics.gmw.cn/2010-10/28/content_13423...,2010-10-28,三、深入实施西部大开发战略，为宁夏招商引资明确了新任务\n今后10年是深入实施西部大开发战略...,光明网,2010Q4,3577,三深入实施西部大开发战略为宁夏招商引资明确了新任务 今后10年是深入实施西部大开发战略的关键...,3.0,3
1,日本外相担忧朝铀浓缩称美日韩应加强情报合作 - 国际新闻,https://world.huanqiu.com/article/9CaKrnJpmLX,2010-11-22,环球网记者李宗泽报道，据韩联社11月22日消息，日本外相前原诚司22日在外务省会见美国国务院...,环球网,2010Q4,313,环球网记者李宗泽报道据韩联社11月22日消息日本外相前原诚司22日在外务省会见美国国务院对朝...,3.0,-2
2,日媒称日美韩外长会谈将发表联合声明对朝施压 - 国际新闻,https://world.huanqiu.com/article/9CaKrnJpxsJ,2010-12-06,环球网记者仲伟东12月6日报道，日本共同社6日报道称，美日韩三国外长将于美国当地时间6日下午...,环球网,2010Q4,464,环球网记者仲伟东12月6日报道日本共同社6日报道称美日韩三国外长将于美国当地时间6日下午在华...,3.0,-2


In [129]:
df["sentiment_score"] = df["input_text"].apply(analyze_sentiment)
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=3) as executor:  # 최대 3개 병렬 요청
    df["sentiment_score"] = list(executor.map(analyze_sentiment_with_delay, df["input_text"]))


InternalServerError: Error code: 500 - {'error': {'message': 'An internal error occurred', 'type': 'server_error', 'param': None, 'code': None}}

In [134]:
df

,title,link,date,content,source,quarter,token_count,input_text,sentiment_scores
0,抢抓新一轮西部大开发战略机遇 全力推进宁夏招商引资新跨越_光明网,https://topics.gmw.cn/2010-10/28/content_13423...,2010-10-28,三、深入实施西部大开发战略，为宁夏招商引资明确了新任务\n今后10年是深入实施西部大开发战略...,光明网,2010Q4,3577,三深入实施西部大开发战略为宁夏招商引资明确了新任务 今后10年是深入实施西部大开发战略的关键...,3.0
1,日本外相担忧朝铀浓缩称美日韩应加强情报合作 - 国际新闻,https://world.huanqiu.com/article/9CaKrnJpmLX,2010-11-22,环球网记者李宗泽报道，据韩联社11月22日消息，日本外相前原诚司22日在外务省会见美国国务院...,环球网,2010Q4,313,环球网记者李宗泽报道据韩联社11月22日消息日本外相前原诚司22日在外务省会见美国国务院对朝...,3.0
2,日媒称日美韩外长会谈将发表联合声明对朝施压 - 国际新闻,https://world.huanqiu.com/article/9CaKrnJpxsJ,2010-12-06,环球网记者仲伟东12月6日报道，日本共同社6日报道称，美日韩三国外长将于美国当地时间6日下午...,环球网,2010Q4,464,环球网记者仲伟东12月6日报道日本共同社6日报道称美日韩三国外长将于美国当地时间6日下午在华...,3.0
3,美国大棋局中的朝鲜半岛危机-博览群书 - 光明数字报,https://epaper.gmw.cn/blqs/html/2011-02/07/nw....,2011-02-07,2010年，“天安号”事件、“延坪岛”事件相继爆发，美韩、美日大规模军演不断，朝鲜更是严阵以...,光明网,2011Q1,6914,2010年天安号事件延坪岛事件相继爆发美韩美日大规模军演不断朝鲜更是严阵以待。朝鲜半岛突然间...,3.0
4,前原诚司在韩国发表演讲称美日韩并非要“封杀”中国 - 国际新闻,https://world.huanqiu.com/article/9CaKrnJsHrU,2011-10-11,环球网记者王欢报道 10月11日上午，正在韩国访问的日本民主党政调会长前原诚司在首尔发表演讲...,环球网,2011Q4,524,环球网记者王欢报道 10月11日上午正在韩国访问的日本民主党政调会长前原诚司在首尔发表演讲。...,3.0
...,...,...,...,...,...,...,...,...,...
460,布林肯访问韩国尹锡悦有救了？还是空欢喜？,http://m.ce.cn/gj/gd/202501/07/t20250107_39258...,2025-01-07,当地时间1月6日，韩国外交部长官赵兑烈与到访的美国国务卿布林肯举行会谈，就韩美同盟等问题展开...,中国经济网,2025Q1,2971,当地时间1月6日韩国外交部长官赵兑烈与到访的美国国务卿布林肯举行会谈就韩美同盟等问题展开磋商...,3.0
461,朝鲜外务省：美韩军事挑衅行为导致地区局势恶化 - 国际频道,https://world.gmw.cn/2025-01/26/content_378203...,2025-01-26,中新网1月26日电据朝中社报道，当地时间1月26日，朝鲜外务省对外政策室长发表谈话，谴责美韩...,光明网,2025Q1,405,中新网1月26日电据朝中社报道当地时间1月26日朝鲜外务省对外政策室长发表谈话谴责美韩近期对...,3.0
462,朝鲜谴责美韩军事挑衅导致地区局势恶化 - 国际经济,http://intl.ce.cn/qqss/202501/26/t20250126_392...,2025-01-26,新华社首尔1月26日电 据朝中社26日报道，朝鲜外务省对外政策室长当天发表谈话，谴责美韩近期...,中国经济网,2025Q1,391,新华社首尔1月26日电 据朝中社26日报道朝鲜外务省对外政策室长当天发表谈话谴责美韩近期对朝...,3.0
463,朝鲜谴责美韩军事挑衅导致地区局势恶化 - 国际频道,https://world.gmw.cn/2025-01/26/content_378203...,2025-01-26,据朝中社1月26日报道，朝鲜外务省对外政策室长当天发表谈话，谴责美韩近期对朝军事挑衅行为导致...,光明网,2025Q1,381,据朝中社1月26日报道朝鲜外务省对外政策室长当天发表谈话谴责美韩近期对朝军事挑衅行为导致地区...,3.0


#### 함수들

In [97]:
import numpy as np
# 지티피 답변에서 숫자만 추출하는 함수
def extract_first_number(text):
    match = re.search(r"-?\d+", text.strip())  # 첫 번째 숫자 찾기 (- 포함 가능)
    return int(match.group()) if match else 0  # 숫자 변환 (없으면 None)

#가중평균계산
def weighted_average(scores):
    positives = [s for s in scores if s > 0]  # 긍정 점수 리스트
    negatives = [s for s in scores if s < 0]  # 부정 점수 리스트

    if len(positives) > len(negatives):  # 긍정 문장이 더 많으면
        return round(np.mean(positives) * 0.7 + np.mean(scores) * 0.3, 2)
    elif len(negatives) > len(positives):  # 부정 문장이 더 많으면
        return round(np.mean(negatives) * 0.7 + np.mean(scores) * 0.3, 2)
    else:  # 긍정과 부정 개수가 비슷하면 일반 평균 반환
        return round(np.mean(scores), 2)

# DataFrame에서 감성 분석 실행
def analyze_sentiment_scores(df):
    def process_row(row):
        input_texts = row['input_text']  # 본문 (리스트 or 단일 문자열)

        # ✅ 리스트일 경우 그대로 사용, 단일 문자열일 경우 리스트로 변환
        input_texts = [input_texts] if isinstance(input_texts, str) else input_texts

        # ✅ Batch API를 사용하여 모든 문장 한 번에 처리
        responses = sentiment_chain.invoke_batch([{"input_text": t} for t in input_texts])
        scores = [extract_first_number(resp.content.strip()) for resp in responses]

        final_score = weighted_average(scores)
        return final_score

    df['sentiment_scores'] = df.apply(process_row, axis=1)
    return df

#### 감성 분석 실행

In [116]:
df = analyze_sentiment_scores(df)


🧐 GPT에 전달되는 단일 문장:
三深入实施西部大开发战略为宁夏招商引资明确了新任务 今后10年是深入实施西部大开发战略的关键时期做好宁夏招商引资工作要认真贯彻落实党中央国务院关于深入实施西部大开发战略的若干意见国务院关于进一步促进宁夏经济社会发展的若干意见精神和自治区党委政府作出的一系列决策部署坚持瞄准产业注重内生突出外引创新突破全面推进自治区招商引资实现新跨越。 一依托新兴经济城市带群的合纵整合着力推进区域间经济技术交流合作。第二轮西部大开发中呼包银经济带银鄂榆能源化工金三角及宁夏沿黄城市带建设带被列入国家层面规划发展给宁夏对外开放区域合作及招商引资带来了重大战略机遇和更加广阔的领域。一是抓住促进呼和浩特包头银川经济区建设的机遇充分发挥银川市在西部地区新的经济增长带中区域中心城市的引领作用主打基础工业先进装备制造业和新材料产业招商牌形成与呼包等地的互动发展。二是抓住促进陕甘宁经济区建设的机遇充分利用国家对革命老区的特殊优惠政策通过对宁夏尤其是南部山区的特色优势产业进行项目储备精品包装塑造品牌打造如中卫枸杞西吉土豆泾源牛肉的招商品牌。三是抓住促进银川鄂尔多斯榆林经济区建设的机遇以煤炭电力煤化工为优势产业为招商重点参与打造国家级能源化工金三角把宁夏建成成为国家重要的能源基地和资源深加工基地。四是抓住宁夏沿黄城市带快速发展的机遇面向世界特别是面向阿拉伯国家着力吸引客商来宁投资置业促进项目落实推进产业导入积极打造内陆开放型经济区扩大对外开放努力将黄河金岸建设成中国与穆斯林世界经济和文化交流的重要平台。 二依托内陆开放型经济的发展着力构建梯次对外开放的空间格局。一是在不断巩固和加强与欧美日韩等国家和地区经贸合作的同时进一步加大宁夏与中东阿拉伯国家交流合作拓展合作领域加大经济交流合作建立内外联动互利共赢安全高效的内陆开放经济体系。二是以宁洽会暨中阿经贸论坛为契机将宁夏着力打造成清真产业和穆斯林用品的设计中心加工中心博览中心物流中心和休闲度假中心具有国际影响力的清真食品穆斯林用品集散地以及承接东部产业转移和阿拉伯国家产业转移的基地面向阿拉伯国家和穆斯林地区合作的前沿阵地向西开放的桥头堡。三是以宁夏银行伊斯兰金融事业部为切入点积极促进发展宁夏伊斯兰金融积极争取国家支持促进我区组建伊斯兰银行发行伊斯兰债券吸引阿盟国家有实力的金融机构来宁设立分支机构或入股地方银行。四是充分利

KeyboardInterrupt: 

In [105]:
import numpy as np
import re

# GPT 응답에서 숫자만 추출하는 함수 (디버깅용)
def extract_first_number(text):
    """GPT 응답에서 첫 번째 숫자 추출"""
    print(f"🔍 GPT 응답 (원본): {text}")  # GPT가 반환하는 원본 응답 출력
    match = re.search(r"-?\d+", text.strip())  # 첫 번째 숫자 찾기 (- 포함 가능)
    score = int(match.group()) if match else None
    print(f"✅ 추출된 점수: {score}")  # 추출된 숫자 확인
    return score

# 가중 평균 계산 함수 (디버깅 추가)
def weighted_average(scores):
    """긍정과 부정 문장의 비율에 따라 가중 평균 계산"""
    print(f"📊 감성 분석 점수 리스트: {scores}")  # 전체 점수 확인
    scores = [s for s in scores if s is not None]  # None 값 제거
    if not scores:
        return None  # 감성 점수가 없으면 None 반환

    positives = [s for s in scores if s > 0]  # 긍정 점수 리스트
    negatives = [s for s in scores if s < 0]  # 부정 점수 리스트

    if len(positives) > len(negatives):  # 긍정 문장이 더 많으면
        final_score = round(np.mean(positives) * 0.7 + np.mean(scores) * 0.3, 2)
    elif len(negatives) > len(positives):  # 부정 문장이 더 많으면
        final_score = round(np.mean(negatives) * 0.7 + np.mean(scores) * 0.3, 2)
    else:  # 긍정과 부정 개수가 비슷하면 일반 평균 반환
        final_score = round(np.mean(scores), 2)

    print(f"🎯 최종 감성 점수: {final_score}")  # 최종 점수 확인
    return final_score
import sys
# 감성 분석 실행
def analyze_sentiment_scores(df):
    def process_row(row):
        input_texts = row['input_text']

        if isinstance(input_texts, list):
            combined_text = " ".join(input_texts)  # ✅ 모든 문장을 하나의 문자열로 결합
            print(f"\n🧐 GPT에 전달되는 병합된 문장:\n{combined_text}")
            response = sentiment_chain.invoke({"input_text": combined_text}).content.strip()
            print(f"🔍 GPT 응답 (원본): {response}")
            final_score = extract_first_number(response)
        else:
            print(f"\n🧐 GPT에 전달되는 단일 문장:\n{input_texts}")
            response = sentiment_chain.invoke({"input_text": input_texts}).content.strip()
            print(f"🔍 GPT 응답 (원본): {response}")
            final_score = extract_first_number(response)

        return final_score

    df['sentiment_scores'] = df.apply(process_row, axis=1)
    return df


In [111]:
print("\n🔍 디버깅: input_text 샘플 (상위 3개)")
df_test = df.head(3).copy()  # 상위 5개 데이터만 복사하여 테스트
df_test = analyze_sentiment_scores(df_test)  # 감성 분석 실행

print("\n📌 감성 분석 결과 (상위 3개):")
print(df_test[['sentiment_scores']])  # 점수 출력



🔍 디버깅: input_text 샘플 (상위 3개)

🧐 GPT에 전달되는 단일 문장:
三深入实施西部大开发战略为宁夏招商引资明确了新任务 今后10年是深入实施西部大开发战略的关键时期做好宁夏招商引资工作要认真贯彻落实党中央国务院关于深入实施西部大开发战略的若干意见国务院关于进一步促进宁夏经济社会发展的若干意见精神和自治区党委政府作出的一系列决策部署坚持瞄准产业注重内生突出外引创新突破全面推进自治区招商引资实现新跨越。 一依托新兴经济城市带群的合纵整合着力推进区域间经济技术交流合作。第二轮西部大开发中呼包银经济带银鄂榆能源化工金三角及宁夏沿黄城市带建设带被列入国家层面规划发展给宁夏对外开放区域合作及招商引资带来了重大战略机遇和更加广阔的领域。一是抓住促进呼和浩特包头银川经济区建设的机遇充分发挥银川市在西部地区新的经济增长带中区域中心城市的引领作用主打基础工业先进装备制造业和新材料产业招商牌形成与呼包等地的互动发展。二是抓住促进陕甘宁经济区建设的机遇充分利用国家对革命老区的特殊优惠政策通过对宁夏尤其是南部山区的特色优势产业进行项目储备精品包装塑造品牌打造如中卫枸杞西吉土豆泾源牛肉的招商品牌。三是抓住促进银川鄂尔多斯榆林经济区建设的机遇以煤炭电力煤化工为优势产业为招商重点参与打造国家级能源化工金三角把宁夏建成成为国家重要的能源基地和资源深加工基地。四是抓住宁夏沿黄城市带快速发展的机遇面向世界特别是面向阿拉伯国家着力吸引客商来宁投资置业促进项目落实推进产业导入积极打造内陆开放型经济区扩大对外开放努力将黄河金岸建设成中国与穆斯林世界经济和文化交流的重要平台。 二依托内陆开放型经济的发展着力构建梯次对外开放的空间格局。一是在不断巩固和加强与欧美日韩等国家和地区经贸合作的同时进一步加大宁夏与中东阿拉伯国家交流合作拓展合作领域加大经济交流合作建立内外联动互利共赢安全高效的内陆开放经济体系。二是以宁洽会暨中阿经贸论坛为契机将宁夏着力打造成清真产业和穆斯林用品的设计中心加工中心博览中心物流中心和休闲度假中心具有国际影响力的清真食品穆斯林用品集散地以及承接东部产业转移和阿拉伯国家产业转移的基地面向阿拉伯国家和穆斯林地区合作的前沿阵地向西开放的桥头堡。三是以宁夏银行伊斯兰金融事业部为切入点积极促进发展宁夏伊斯兰金融积极争取国家支持促进我区组建伊斯兰银行发行伊斯兰债券吸引阿盟国

In [113]:
df

,title,link,date,content,source,quarter,token_count,input_text,sentiment_scores
0,抢抓新一轮西部大开发战略机遇 全力推进宁夏招商引资新跨越_光明网,https://topics.gmw.cn/2010-10/28/content_13423...,2010-10-28,三、深入实施西部大开发战略，为宁夏招商引资明确了新任务\n今后10年是深入实施西部大开发战略...,光明网,2010Q4,3577,三深入实施西部大开发战略为宁夏招商引资明确了新任务 今后10年是深入实施西部大开发战略的关键...,3.0
1,日本外相担忧朝铀浓缩称美日韩应加强情报合作 - 国际新闻,https://world.huanqiu.com/article/9CaKrnJpmLX,2010-11-22,环球网记者李宗泽报道，据韩联社11月22日消息，日本外相前原诚司22日在外务省会见美国国务院...,环球网,2010Q4,313,环球网记者李宗泽报道据韩联社11月22日消息日本外相前原诚司22日在外务省会见美国国务院对朝...,3.0
2,日媒称日美韩外长会谈将发表联合声明对朝施压 - 国际新闻,https://world.huanqiu.com/article/9CaKrnJpxsJ,2010-12-06,环球网记者仲伟东12月6日报道，日本共同社6日报道称，美日韩三国外长将于美国当地时间6日下午...,环球网,2010Q4,464,环球网记者仲伟东12月6日报道日本共同社6日报道称美日韩三国外长将于美国当地时间6日下午在华...,3.0
3,美国大棋局中的朝鲜半岛危机-博览群书 - 光明数字报,https://epaper.gmw.cn/blqs/html/2011-02/07/nw....,2011-02-07,2010年，“天安号”事件、“延坪岛”事件相继爆发，美韩、美日大规模军演不断，朝鲜更是严阵以...,光明网,2011Q1,6914,2010年天安号事件延坪岛事件相继爆发美韩美日大规模军演不断朝鲜更是严阵以待。朝鲜半岛突然间...,3.0
4,前原诚司在韩国发表演讲称美日韩并非要“封杀”中国 - 国际新闻,https://world.huanqiu.com/article/9CaKrnJsHrU,2011-10-11,环球网记者王欢报道 10月11日上午，正在韩国访问的日本民主党政调会长前原诚司在首尔发表演讲...,环球网,2011Q4,524,环球网记者王欢报道 10月11日上午正在韩国访问的日本民主党政调会长前原诚司在首尔发表演讲。...,3.0
...,...,...,...,...,...,...,...,...,...
460,布林肯访问韩国尹锡悦有救了？还是空欢喜？,http://m.ce.cn/gj/gd/202501/07/t20250107_39258...,2025-01-07,当地时间1月6日，韩国外交部长官赵兑烈与到访的美国国务卿布林肯举行会谈，就韩美同盟等问题展开...,中国经济网,2025Q1,2971,当地时间1月6日韩国外交部长官赵兑烈与到访的美国国务卿布林肯举行会谈就韩美同盟等问题展开磋商...,3.0
461,朝鲜外务省：美韩军事挑衅行为导致地区局势恶化 - 国际频道,https://world.gmw.cn/2025-01/26/content_378203...,2025-01-26,中新网1月26日电据朝中社报道，当地时间1月26日，朝鲜外务省对外政策室长发表谈话，谴责美韩...,光明网,2025Q1,405,中新网1月26日电据朝中社报道当地时间1月26日朝鲜外务省对外政策室长发表谈话谴责美韩近期对...,3.0
462,朝鲜谴责美韩军事挑衅导致地区局势恶化 - 国际经济,http://intl.ce.cn/qqss/202501/26/t20250126_392...,2025-01-26,新华社首尔1月26日电 据朝中社26日报道，朝鲜外务省对外政策室长当天发表谈话，谴责美韩近期...,中国经济网,2025Q1,391,新华社首尔1月26日电 据朝中社26日报道朝鲜外务省对外政策室长当天发表谈话谴责美韩近期对朝...,3.0
463,朝鲜谴责美韩军事挑衅导致地区局势恶化 - 国际频道,https://world.gmw.cn/2025-01/26/content_378203...,2025-01-26,据朝中社1月26日报道，朝鲜外务省对外政策室长当天发表谈话，谴责美韩近期对朝军事挑衅行为导致...,光明网,2025Q1,381,据朝中社1月26日报道朝鲜外务省对外政策室长当天发表谈话谴责美韩近期对朝军事挑衅行为导致地区...,3.0
